In [1]:
import os
import subprocess
import re

# RQ2: Do applications request sensitive permissions along with access to internet?

What are sensitive permissions:

- accessing contacts, camera, microphone, location, SMS, call logs, storage

Here is what they are in the manifest:

- `android.permission.READ_CONTACTS`
- `android.permission.CAMERA`
- `android.permission.RECORD_AUDIO`
- `android.permission.ACCESS_FINE_LOCATION`
- `android.permission.ACCESS_COARSE_LOCATION`
- `android.permission.READ_SMS`
- `android.permission.READ_CALL_LOG`
- `android.permission.WRITE_EXTERNAL_STORAGE`
- `android.permission.READ_EXTERNAL_STORAGE`
- `android.permission.WRITE_CONTACTS`
- `android.permission.CALL_PHONE`
- `android.permission.SEND_SMS`

In [2]:
output_path = os.path.join("decoded_apks", "AC-Remote-Control_2.1_Apkpure")
package_name = 'com.daimler.mm.android'
manifest_path = os.path.join(output_path, 'AndroidManifest.xml')

In [3]:
def extract_package_name(manifest_file_path: str):
    with open(manifest_file_path, 'r') as f:
        content = f.read()
        return content

In [4]:
content = extract_package_name(manifest_path)
match = re.search(r'package="(.+?)"', content)
match.groups()[0]

'com.ac.remote.control.air.conditioner.temperature'

In [8]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [8]:
sensitive_permissions = [
    'android.permission.CAMERA',
    'android.permission.RECORD_AUDIO',
    'android.permission.READ_SMS',
    'android.permission.READ_CONTACTS',
    'android.permission.READ_PHONE_STATE',
    'android.permission.READ_CALL_LOG',
    'android.permission.READ_EXTERNAL_STORAGE',
    'android.permission.WRITE_EXTERNAL_STORAGE',
    'android.permission.ACCESS_FINE_LOCATION',
    'android.permission.ACCESS_COARSE_LOCATION',
    'android.permission.CALL_PHONE',
    'android.permission.SEND_SMS',
]

In [10]:
def check_has_internet_permission():
    manifest_content = read_file(manifest_path)
    if 'android.permission.INTERNET' in manifest_content:
        return True
    return False

In [11]:
def tally_sensitive_permissions():
    manifest_content = read_file(manifest_path)
    permissions = re.findall(r'<uses-permission android:name="([^"]+)"', manifest_content)
    sensitive_permissions_count = 0
    permissions_list = []
    for permission in permissions:
        if permission in sensitive_permissions:
            # print(f"found sensitive permission: {permission}")
            permissions_list.append(permission)
            sensitive_permissions_count += 1

    permissions_dict = { permission: 1 for permission in permissions_list }
    return sensitive_permissions_count, permissions_dict

In [26]:
from utilities import Markdowner, JSONPermissions, OSUtils

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'apps/MyPlate-Calorie-Tracker_3.5.4(4)_Apkpure.apk' -> 'apps/MyPlate-Calorie-Tracker_3.5.4(4-_Apkpure.apk'

In [13]:
hasInternetPermission = check_has_internet_permission()
if hasInternetPermission:
    sensitive_permissions_count, permissions_dict = tally_sensitive_permissions()
    json_permissions = JSONPermissions()
    json_permissions.write(permissions_dict)
    json_permissions.addInternet()

In [3]:
import pyperclip

In [12]:
def create_list(arr, mode = 'markdown'):
    if mode == 'markdown':
        strtocopy = ""
        for item in arr:
            strtocopy += f"- {item}\n"
        pyperclip.copy(strtocopy)
        return strtocopy
    elif mode == 'latex':
        strtocopy = "\\begin{itemize}\n"
        for item in arr:
            strtocopy += f"\t\\item \\verb+{item}+ \n\n"
        strtocopy += "\\end{itemize}"
        pyperclip.copy(strtocopy)
        return strtocopy

In [13]:
create_list(sensitive_permissions, mode='latex')

'\\begin{itemize}\n\t\\item \\verb+android.permission.CAMERA+ \n\n\t\\item \\verb+android.permission.RECORD_AUDIO+ \n\n\t\\item \\verb+android.permission.READ_SMS+ \n\n\t\\item \\verb+android.permission.READ_CONTACTS+ \n\n\t\\item \\verb+android.permission.READ_PHONE_STATE+ \n\n\t\\item \\verb+android.permission.READ_CALL_LOG+ \n\n\t\\item \\verb+android.permission.READ_EXTERNAL_STORAGE+ \n\n\t\\item \\verb+android.permission.WRITE_EXTERNAL_STORAGE+ \n\n\t\\item \\verb+android.permission.ACCESS_FINE_LOCATION+ \n\n\t\\item \\verb+android.permission.ACCESS_COARSE_LOCATION+ \n\n\t\\item \\verb+android.permission.CALL_PHONE+ \n\n\t\\item \\verb+android.permission.SEND_SMS+ \n\n\\end{itemize}'

### My findings

Here is the application permission count: 

```json
{
  "android.permission.CAMERA": 48,
  "android.permission.RECORD_AUDIO": 28,
  "android.permission.READ_SMS": 7,
  "android.permission.READ_CONTACTS": 20,
  "android.permission.READ_PHONE_STATE": 32,
  "android.permission.READ_CALL_LOG": 12,
  "android.permission.READ_EXTERNAL_STORAGE": 59,
  "android.permission.WRITE_EXTERNAL_STORAGE": 65,
  "android.permission.ACCESS_FINE_LOCATION": 53,
  "android.permission.ACCESS_COARSE_LOCATION": 52,
  "android.permission.CALL_PHONE": 10,
  "android.permission.SEND_SMS": 6,
  "android.permission.INTERNET": 97
}
```

Of the 99 apps we analyzed, 97 requested internet access. This is not surprising as most apps need internet access to function.

# RQ4: Do applications override the AllowAllHostnameVerifier interface incorrectly?

In [2]:
from utilities import OSUtils, Markdowner, JSONCreator
import datetime

In [4]:
date = datetime.datetime.now().strftime("%m-%d-%Y %H-%M")
markdowner = Markdowner(f"results/output-rq4-{date}.md")

global_paths = {
    "APK_DIR": "apps",
    "DECODED_APK_DIR": "decoded_apks",
    "output_path": None,
    "manifest_file_path": None,
    "package_name": None,
    "app_dir": None
}

def extract_package_name(manifest_file_path: str):
    print(f"extracting package name from AndroidManifest.xml file at {manifest_file_path} ...")
    with open(manifest_file_path, 'r') as f:
        content = f.read()
        match = re.search(r'package="(.+?)"', content, flags=re.IGNORECASE)
        package_name = match.groups()[0]
    return package_name

In [15]:
def check_for_hostname_verifier(filepath, content):
    # print("Checking for AllowAllHostnameVerifier in the code...")
    vulnerabilities = 0
    def add_file_link(file_path , line_number):
        return  f"[Go to file]({file_path}#L{line_number})"
    
    if "AllowAllHostnameVerifier" in content:
        print(f"AllowAllHostnameVerifier found in file: {filepath}")
        markdowner.write_heading(f"`AllowAllHostnameVerifier` vulnerabilities found in file: `{filepath}`\n", level=3)

        for match in re.finditer(r'.*AllowAllHostnameVerifier.*', 
                                 content, 
                                 flags=re.MULTILINE):
            line_number = content.count("\n", 0, match.start()) + 1
            text = match.group()
            markdowner.write_bullet(f"""found AllowAllHostnameVerifier at line {line_number}. {add_file_link(filepath, line_number)}\n\n```smali\n{text}\n```\n""")
            vulnerabilities += 1
    return vulnerabilities

def extract_vulnerability():
    app_dir = global_paths["app_dir"]
    vulnerability_count = 0
    for root, _, files in os.walk(app_dir):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                content = f.read()
                vulnerability_count = check_for_hostname_verifier(file_path, content)
    if vulnerability_count == 0:
        return None
    return { app_dir: vulnerability_count }

In [5]:
jsoner = JSONCreator("results/rq4.json")

In [16]:
vulnerabilities = []

for foldername in os.listdir(global_paths["DECODED_APK_DIR"]):
    global_paths["output_path"] = os.path.join(global_paths["DECODED_APK_DIR"], foldername)
    global_paths["manifest_file_path"] = os.path.join(global_paths["output_path"], "AndroidManifest.xml")
    if not os.path.exists(global_paths["manifest_file_path"]):
        print(f"AndroidManifest.xml file not found at {global_paths['manifest_file_path']}")
        continue
    global_paths["package_name"] = extract_package_name(global_paths["manifest_file_path"])
    package_name = global_paths["package_name"]
    app_dir = os.path.join(global_paths["output_path"], "smali", package_name.replace(".", "/"))
    global_paths["app_dir"] = app_dir

    obj = extract_vulnerability()
    if obj:
        vulnerabilities.append(obj)

jsoner.write(vulnerabilities)

extracting package name from AndroidManifest.xml file at decoded_apks\AC-Remote-Control_2.1_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\AC-Remote-For-LG-Air-Condition_10.0.0.1_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\AC-Remote-For-Panasonic_10.0.0.1_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\AIO-Remote_3.5.9.14_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\Amazon-Alexa_2.2.486074.0_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\Arçelik-Akıllı-Kumanda_3.36_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\Bike-Itaú_-Bicycle-Sharing_9.3.2_Apkpure\AndroidManifest.xml ...
extracting package name from AndroidManifest.xml file at decoded_apks\Bird-—-Ride-El

FileNotFoundError: [Errno 2] No such file or directory: 'decoded_apks\\Cast-to-Chromecast---Screen-mi_2.2_Apkpure\\smali\\chromecast/tv/cast/caster\\audio\\ui\\component\\folder\\audio\\AudioFolderFragment$initUpdateAdsTimer$1$$ExternalSyntheticLambda0.smali'

In [8]:
appdir = "decoded_apks/Bike-Itaú_-Bicycle-Sharing_9.3.2_Apkpure/smali/pbsc/cyclefinder/tembici"
vulnerability_count = 0
for root, _, files in os.walk(appdir):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)
        with open(file_path, 'r') as f:
            content = f.read()
            vulnerability_count = check_for_hostname_verifier(file_path, content)
vulnerability_count

0

In [1]:
import os
from androguard.core.analysis.analysis import Analysis
from androguard.core.bytecode import get_package_class_name, FormatClassToPython
from androguard.core import dex, apk
from androguard.core.apk import APK
from androguard.decompiler.decompile import DvClass
from androguard.misc import AnalyzeAPK
from utilities import OSUtils, Markdowner, JSONCreator

def _get_java_code(_class, _vmx):
    try:
        _ms = DvClass(_class, _vmx)
        _ms.process()
        return _ms.get_source()
    except Exception as e:
        print("Error getting Java source code for: {:s}".format(_class.get_name()))
    return None

In [2]:
def run_hostname_verifier_check(apk_path: str):
    markdowner = Markdowner(f"results/rq4.md")
    markdowner.clear()
    markdowner.write_heading("RQ4: Hostname Verifier Vulnerabilities", level=1)
    a, d, dx = AnalyzeAPK(apk_path)
    # apk = APK(apk_path)
    # package_name = apk.package
    # permissions = apk.get_permissions()
    # markdowner.write_heading(f"App name: {apk_path} Package Name: {package_name}", level=2)


In [5]:
run_hostname_verifier_check("apps/Thermometer-Free_1.6.1_Apkpure.apk")

2024-05-10 13:41:18.542 | DEBUG    | androguard.misc:AnalyzeAPK:41 - AnalyzeAPK
2024-05-10 13:41:18.544 | DEBUG    | androguard.misc:AnalyzeAPK:56 - Analysing without session
2024-05-10 13:41:18.594 | INFO     | androguard.core.apk:_apk_analysis:312 - Starting analysis on AndroidManifest.xml
2024-05-10 13:41:18.596 | DEBUG    | androguard.core.axml:__init__:940 - AXMLPrinter
2024-05-10 13:41:18.597 | DEBUG    | androguard.core.axml:__init__:375 - AXMLParser
2024-05-10 13:41:18.598 | DEBUG    | androguard.core.axml:__init__:399 - FIRST HEADER <ARSCHeader idx='0x00000000' type='3' header_size='8' size='27764'>
2024-05-10 13:41:18.599 | DEBUG    | androguard.core.axml:__init__:439 - STRING_POOL <ARSCHeader idx='0x00000008' type='1' header_size='28' size='13124'>
2024-05-10 13:41:18.600 | DEBUG    | androguard.core.axml:is_valid:466 - True
2024-05-10 13:41:18.601 | DEBUG    | androguard.core.axml:_do_next:484 - M_EVENT -1
2024-05-10 13:41:18.602 | DEBUG    | androguard.core.axml:_do_next:5